# SPR 2026 - Mistral 7B Instruct (One-Shot Classification) v2

**Estratégia:** One-Shot com 1 exemplo por classe no prompt

**Correção v2:** Adicionado instalação de `bitsandbytes` para quantização 4-bit

**Modelo:** Mistral 7B é um modelo de alta qualidade com bom desempenho em tarefas de NLP.

---
**CONFIGURAÇÃO KAGGLE:**

1. **Add Input** → **Competition** → `spr-2026-mammography-report-classification`
2. **Add Input** → **Models** → buscar `Mistral 7B`
   - Autor: **MistralAI**
   - Variation: **transformers** (NÃO GGUF!)
3. **Settings** → Internet → **OFF**
4. **Settings** → Accelerator → **GPU T4 x2**

---

In [ ]:
# Instalar bitsandbytes para quantização 4-bit
!pip install -q bitsandbytes>=0.46.1
print("bitsandbytes instalado!")

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("SPR 2026 - Mistral 7B Instruct One-Shot Classification v2")
print("="*60)

SEED = 42
DATA_DIR = '/kaggle/input/competitions/spr-2026-mammography-report-classification'
USE_4BIT = True  # Quantização 4-bit para caber na GPU

# Verificar dataset
if not os.path.exists(DATA_DIR):
    print("\n" + "="*60)
    print("ERRO: Dataset não encontrado!")
    print("="*60)
    print("\nAdicione o dataset:")
    print("Add Input → Competition → spr-2026-mammography-report-classification")
    raise FileNotFoundError(f"Dataset não encontrado: {DATA_DIR}")

# Auto-detectar modelo HuggingFace
def find_model():
    base = '/kaggle/input'
    hf_path = None
    def search_dir(directory, depth=0, max_depth=8):
        nonlocal hf_path
        if depth > max_depth:
            return
        try:
            for item in os.listdir(directory):
                path = os.path.join(directory, item)
                if os.path.isdir(path):
                    if os.path.exists(os.path.join(path, 'config.json')):
                        hf_path = path
                        return
                    search_dir(path, depth + 1, max_depth)
        except:
            pass
    search_dir(base)
    return hf_path

MODEL_PATH = find_model()
if MODEL_PATH is None:
    print("\nModelo não encontrado!")
    print("Adicione: MistralAI/Mistral-7B-Instruct (variação transformers)")
    raise FileNotFoundError("Modelo Mistral 7B não encontrado")

USE_GPU = torch.cuda.is_available()
device = 'cuda' if USE_GPU else 'cpu'

print(f"Modelo: {MODEL_PATH}")
print(f"GPU: {USE_GPU}")
print(f"Device: {device}")
print(f"4-bit: {USE_4BIT}")

In [ ]:
# Carregar dados
print("\n[1/5] Carregando dados...")
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')
print(f"Train: {train.shape} | Test: {test.shape}")

# Auto-detectar colunas
TEXT_COL = None
for col in ['report', 'text', 'laudo', 'texto']:
    if col in train.columns:
        TEXT_COL = col
        break

LABEL_COL = None
for col in ['target', 'label', 'birads']:
    if col in train.columns:
        LABEL_COL = col
        break

ID_COL = None
for col in ['ID', 'id', 'Id']:
    if col in test.columns:
        ID_COL = col
        break

print(f"Colunas: texto={TEXT_COL}, label={LABEL_COL}, id={ID_COL}")

In [ ]:
# Selecionar exemplos para one-shot (1 por classe)
print("\n[2/5] Selecionando exemplos para one-shot...")

examples = {}
for classe in sorted(train[LABEL_COL].unique()):
    # Pegar um exemplo curto e claro de cada classe
    exemplos_classe = train[train[LABEL_COL] == classe]
    # Ordenar por tamanho (preferir exemplos mais curtos)
    exemplos_classe = exemplos_classe.copy()
    exemplos_classe['len'] = exemplos_classe[TEXT_COL].str.len()
    exemplos_classe = exemplos_classe.sort_values('len')
    # Pegar um do meio (nem muito curto nem muito longo)
    idx = len(exemplos_classe) // 3
    exemplo = exemplos_classe.iloc[idx][TEXT_COL]
    examples[classe] = exemplo[:500]  # Limitar tamanho
    print(f"Classe {classe}: {len(examples[classe])} chars")

print(f"\nTotal: {len(examples)} exemplos selecionados")

In [ ]:
# Carregar modelo com quantização 4-bit
print("\n[3/5] Carregando Mistral 7B...")

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

if USE_4BIT and USE_GPU:
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        quantization_config=quantization_config,
        device_map='auto',
        trust_remote_code=True,
        local_files_only=True,
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        torch_dtype=torch.float16 if USE_GPU else torch.float32,
        device_map='auto' if USE_GPU else None,
        trust_remote_code=True,
        local_files_only=True,
    )

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True, local_files_only=True)
model.eval()
print("Modelo carregado!")

In [ ]:
# Prompt template com one-shot usando formato Mistral
ONE_SHOT_PROMPT = """<s>[INST] Você é um radiologista especialista em mamografia.
Classifique o laudo de mamografia na categoria BI-RADS correta (0-6).

Exemplos de cada categoria:

BI-RADS 0 (Inconclusivo):
"{ex0}"

BI-RADS 1 (Negativo):
"{ex1}"

BI-RADS 2 (Benigno):
"{ex2}"

BI-RADS 3 (Provavelmente benigno):
"{ex3}"

BI-RADS 4 (Suspeito):
"{ex4}"

BI-RADS 5 (Altamente suspeito):
"{ex5}"

BI-RADS 6 (Malignidade confirmada):
"{ex6}"

Agora classifique este laudo:
"{text}"

Responda APENAS com o número da categoria (0, 1, 2, 3, 4, 5 ou 6): [/INST]"""

def classify_text(text, max_length=2048):
    """Classifica um laudo usando one-shot."""
    prompt = ONE_SHOT_PROMPT.format(
        ex0=examples.get(0, "N/A")[:200],
        ex1=examples.get(1, "N/A")[:200],
        ex2=examples.get(2, "N/A")[:200],
        ex3=examples.get(3, "N/A")[:200],
        ex4=examples.get(4, "N/A")[:200],
        ex5=examples.get(5, "N/A")[:200],
        ex6=examples.get(6, "N/A")[:200],
        text=str(text)[:500]
    )
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_length)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=3,
            temperature=0.1,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    
    # Extrair número
    for char in response.strip():
        if char.isdigit() and char in '0123456':
            return int(char)
    
    return 2  # Default para classe mais comum

In [ ]:
# Classificar teste
print("\n[4/5] Classificando laudos de teste...")
from tqdm import tqdm

predictions = []
for i, text in enumerate(tqdm(test[TEXT_COL], desc="Classificando")):
    pred = classify_text(text)
    predictions.append(pred)
    
    if i < 3:
        print(f"\nExemplo {i+1}:")
        print(f"Texto: {str(text)[:100]}...")
        print(f"Predição: {pred}")

print(f"\nTotal predições: {len(predictions)}")
print(f"Distribuição: {pd.Series(predictions).value_counts().sort_index().to_dict()}")

In [ ]:
# Gerar submission
print("\n[5/5] Gerando submission...")

submission = pd.DataFrame({
    ID_COL: test[ID_COL],
    LABEL_COL: predictions
})

submission.to_csv('submission.csv', index=False)
print(f"\nSubmission salva: submission.csv")
print(submission.head())
print(f"\nDistribuição final:")
print(submission[LABEL_COL].value_counts().sort_index())